#  Optimisation

Energy Systems 

TU Berlin
***

In [1]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, NonNegativeReals, Suffix
from pyomo.opt import SolverFactory
import pandas as pd

# Exercise 3 c)
> **Remarks:** For time reasons, you do not have to build whole model from scratch. However, we have omitted a few elements by three question marks `???`.

## Set up parameters for the optimization model
Define DataFrame for the two generators gas and coal

In [2]:
gens = ["gas","coal"]

data = pd.DataFrame(index=gens)

data["costs"] = [20,8]
data["capacities"] = [350,300]
data["efficiencies"] = [0.6,0.4]
data["emissions"] = [0.2,0.3]

data

,costs,capacities,efficiencies,emissions
gas,20,350,0.6,0.2
coal,8,300,0.4,0.3


Convert fuel costs and fuel emissions to cost and emission per electrcity unit

In [3]:
data["costs_el"] = data.costs/data.efficiencies
data.costs_el

gas     33.333333
coal    20.000000
Name: costs_el, dtype: float64

In [4]:
data["emissions_el"] = data.emissions/data.efficiencies
data.emissions_el

gas     0.333333
coal    0.750000
Name: emissions_el, dtype: float64

Define demand and CO2 limit

In [5]:
demand = 500
co2_limit = 250

## Create the optimization model 

In [6]:
model = ConcreteModel()

Define variable x for the installed capacity of generators gas and coal

In [7]:
model.x = Var(gens, domain=NonNegativeReals)

Define the power balance constraint that generation equals demand

In [8]:
model.balance = Constraint(expr = model.x["gas"] + model.x["coal"] == demand)

Define additional constraints for capacity limit and CO2 limit

In [9]:
def cap_limits(model,gen):
    return model.x[gen] <= data.capacities[gen]

model.cap_limits = Constraint(gens,rule=cap_limits)

model.emissions = Constraint(expr = sum([data.emissions_el[gen]*model.x[gen] for gen in gens]) <= co2_limit)

Define objective function of the model

In [10]:
model.objective = Objective(expr = sum([data.costs_el[gen]*model.x[gen] for gen in gens]))

Define the solver for the model

In [11]:
opt = SolverFactory("glpk")

In [12]:
model.dual = Suffix(direction=Suffix.IMPORT_EXPORT)

Solve the model and print results

In [13]:
results = opt.solve(model,suffixes=["dual"])

In [14]:
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 14000.0
  Upper bound: 14000.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 3
  Number of nonzeros: 7
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.0060901641845703125
# ----------------------------------------------------------
#   Solution Information
# -------------------------------

Print the built capacities for the generators

In [15]:
for gen in gens:
    print(gen,model.x[gen].value)

gas 300.0
coal 200.0


Print the shadow prices

In [16]:
model.dual[model.balance]

44.0

The dual variable of the balance constraint represents the markets price.

In [17]:
model.dual[model.emissions]

-32.0

The dual variable of the emission constraint represents the marginal abatement cost for CO2 reduction.